In [86]:
import random 
import simpy 
import numpy as np

# Call center simulation

https://youtu.be/8SLk_uRRcgc 

In [87]:
# parameters for simulation
NUM_EMPLOYEES = 2
AVG_SUPPORT_TIME = 5
CUSTOMER_INTERVAL = 2
SIM_TIME = 120

# counter
customers_handled = 0

In [88]:
class CallCenter: 

    def __init__(self, env, num_employees, support_time): #env for simulation, from simpy
        self.env = env
        self.staff = simpy.Resource(env, num_employees) # set number of employees as a resource
        self.support_time = support_time

    def support(self, customer):
        random_time = max(1, np.random.normal(self.support_time, 4)) 
        yield self.env.timeout(random_time)
        print(f"Support fininshed for {customer} at {self.env.now:.2f}")

In [89]:
def customer(env, name, call_center):
    global customers_handled
    print(f"Customer {name} enters waiting queue at {env.now:.2f}")
    with call_center.staff.request() as request:
        yield request
        print(f"Customer {name} enters call at {env.now:.2f}")
        yield env.process(call_center.support(name))
        print(f"Customer {name} left call at {env.now:.2f}")
        customers_handled += 1

In [90]:
def setup(env, num_employees, support_time, customer_interval):
    call_center = CallCenter(env, num_employees, support_time)

    for i in range(1,6):
        env.process(customer(env, num_employees, call_center))

    while True: # while True is True --> loop forever 
        yield env.timeout(random.randint(customer_interval-1, customer_interval+1))
        i += 1
        env.process(customer(env, i, call_center))

In [91]:
print("Starting Call Center Simulation")
env = simpy.Environment()
env.process(setup(env, NUM_EMPLOYEES, AVG_SUPPORT_TIME, CUSTOMER_INTERVAL))
env.run(until=SIM_TIME)

print(f"Customer handled: {customers_handled}")

Starting Call Center Simulation
Customer 2 enters waiting queue at 0.00
Customer 2 enters waiting queue at 0.00
Customer 2 enters waiting queue at 0.00
Customer 2 enters waiting queue at 0.00
Customer 2 enters waiting queue at 0.00
Customer 2 enters call at 0.00
Customer 2 enters call at 0.00
Customer 6 enters waiting queue at 1.00
Customer 7 enters waiting queue at 3.00
Support fininshed for 2 at 4.67
Customer 2 left call at 4.67
Customer 2 enters call at 4.67
Customer 8 enters waiting queue at 5.00
Support fininshed for 2 at 5.67
Customer 2 left call at 5.67
Customer 2 enters call at 5.67
Customer 9 enters waiting queue at 6.00
Customer 10 enters waiting queue at 7.00
Customer 11 enters waiting queue at 8.00
Customer 12 enters waiting queue at 9.00
Customer 13 enters waiting queue at 12.00
Support fininshed for 2 at 12.79
Customer 2 left call at 12.79
Customer 2 enters call at 12.79
Support fininshed for 2 at 12.82
Customer 2 left call at 12.82
Customer 6 enters call at 12.82
Custome

# Understanding generators

2 ways of creating generators:
1. Generator functions
2. Generator expressions

https://realpython.com/introduction-to-python-generators/

## Generator functions

Generator functions look and act just like regular functions, but with one defining characteristic. Generator functions use the Python yield keyword instead of return.

In [92]:
def infinite_sequence():
    num = 0
    while True:
        yield num
        num += 1

inf = infinite_sequence()
print(next(inf))
print(next(inf))

0
1


`yield` indicates where a value is sent back to the caller, but unlike `return`, you *don’t exit the function afterward*.

Instead, the state of the function is *remembered*. That way, when next() is called on a generator object (either explicitly or implicitly within a for loop), the previously yielded variable num is incremented, and then yielded again

## Generator expressions

Like list comprehensions, generator expressions allow you to quickly create a generator object in just a few lines of code. They’re also useful in the same cases where list comprehensions are used, with an added benefit: you can create them without building and holding the entire object in memory before iteration. In other words, you’ll have *no memory penalty* when you use generator expressions

In [93]:
nums_squared_lc = [num**2 for num in range(5)] # creates list using brackets
nums_squared_gc = (num**2 for num in range(5)) # creates generator expression using parentheses

In [94]:
print(nums_squared_lc)
print(nums_squared_gc)

[0, 1, 4, 9, 16]
<generator object <genexpr> at 0x00000256FD0EB270>


# Event discrete simulation with SimPy

https://www.youtube.com/watch?v=Bk91DoAEcjY

In [95]:
def clock(env, name, tick):
    while True:
        # clock will repeat printing current simulation time
        print(name, env.now)
        # stop the clock
        yield env.timeout(tick)

# create environment
env = simpy.Environment() 

# print every half simulation step
env.process(clock(env, 'fast', 0.5))

# print every simulation step
env.process(clock(env, 'slow', 1))

# run until simulation time 2
env.run(until=2)

fast 0
slow 0
fast 0.5
slow 1
fast 1.0
fast 1.5


There are 2 kinds of environments:
1. Environment - simulate "as fast as possible"
2. RealtimeEnvionment - synchronized with wall-clock time
    * Observe real-time behaviour

In [96]:
# timeout events let time pass

def speaker(env, start):
    while True:
        until_start = start - env.now

        # let time pass to wait until the start of the talk
        yield env.timeout(until_start)

        # end the talk after 30 minutes
        yield env.timeout(30)

        # let next speaker start after prev speaker ends
        start = env.now

        # print end timings
        print(env.now)

env = simpy.Environment()
env.process(speaker(env, 10))
env.run(until=100)

40
70


In [97]:
# processes are events too

def speaker(env):
    # end the talk after 30 minutes
    yield env.timeout(30)
    return('end of talk', env.now)

def moderator(env):
    # 3 speakers
    for i in range(3):
        # spawn speaker process and wait for that to end before sending another speaker
        val = yield env.process(speaker(env))
        print(val)

env = simpy.Environment()
env.process(moderator(env))
env.run(until=100)

('end of talk', 30)
('end of talk', 60)
('end of talk', 90)


## Asynchronous interrupts

Documentation: https://simpy.readthedocs.io/en/latest/topical_guides/resources.html


In reality, not all speakers speak for exactly 30 minutes. Moderators must interrupt speakers if they exceed their time limit

In [98]:
from random import randint

def speaker(env):
    try:
        talk_duration = randint(25, 35)
        yield env.timeout(talk_duration)
        print(talk_duration)
    except simpy.Interrupt as interrupt:
        print(interrupt.cause)

def moderator(env):
    for i in range(3):
        speaker_proc = env.process(speaker(env))
        results = yield speaker_proc | env.timeout(30) 
        print(results)

        if speaker_proc not in results:
            speaker_proc.interrupt('No time left')

env = simpy.Environment()
env.process(moderator(env))
env.run()

27
<ConditionValue {<Process(speaker) object at 0x256fd12fd60>: None}>
25
<ConditionValue {<Process(speaker) object at 0x256fd12faf0>: None}>
<ConditionValue {<Timeout(30) object at 0x256fd12fd00>: None}>
No time left


## Shared resources 

Documentation: https://simpy.readthedocs.io/en/latest/topical_guides/resources.html

Shared resources are another way to model Process Interaction. They form a congestion point where processes *queue* up in order to use them.

3 categories:
1. Resource - Resources that can be used by a limited number of processes at a time (e.g., a gas station with a limited number of fuel pumps).
2. Store - Resources that allow the production and consumption of Python objects.
3. Container - Resources that model the production and consumption of a homogeneous, undifferentiated bulk. It may either be continuous (like water) or discrete (like apples).

All resources share the same basic concept: The resource itself is some kind of a container with a, usually limited, capacity. Processes can either try to put something into the resource or try to get something out. If the resource is full or empty, they have to queue up and wait.

Requesting a resource is modeled as “putting a process’ token into the resource” and releasing a resource correspondingly as “getting a process’ token out of the resource”. Thus, calling `request()`/`release()` is equivalent to calling `put()`/`get()`. Releasing a resource will always succeed immediately.

## Resource

https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#resource

The Resource is conceptually a semaphore. Its only parameter – apart from the obligatory reference to an Environment – is its *capacity*. It must be a positive number and defaults to 1: `Resource(env, capacity=1)`

In [99]:
# imports
from random import randint 
import simpy

Now, we will use simpy to model conference attendees. We will model a continuous flow of talks sessions, with each session containing 3 talks of 30 mins each. There will be a 15 mins break between each session (3 talks)

In [100]:
# config
TALKS_PER_SESSION = 3
TALK_LENGTH = 30
BREAK_LENGTH = 15

In [101]:
def attendee(env, name, knowledge=0, hunger=0):
    # repeat sessions
    while True:
        # visit talks
        for i in range(TALKS_PER_SESSION):
            knowledge += randint(0,3) / (1+hunger) # the more hungry the less knowledge
            hunger += randint(1,4)

            yield env.timeout(TALK_LENGTH)

        print(f'Attendee {name} finished talks with knowledge {knowledge:.2f} and hunger {hunger}')

        # go to buffet
        food = randint(3,12)
        hunger -= min(food, hunger)

        yield env.timeout(BREAK_LENGTH)

        print(f"Attendee {name} finished eating with hunger {hunger}")

# run simulation
env = simpy.Environment()
for i in range(5): 
    env.process(attendee(env, i))
env.run(until=220)

Attendee 0 finished talks with knowledge 1.40 and hunger 8
Attendee 1 finished talks with knowledge 1.25 and hunger 7
Attendee 2 finished talks with knowledge 3.17 and hunger 8
Attendee 3 finished talks with knowledge 1.29 and hunger 9
Attendee 4 finished talks with knowledge 0.50 and hunger 8
Attendee 0 finished eating with hunger 3
Attendee 1 finished eating with hunger 0
Attendee 2 finished eating with hunger 3
Attendee 3 finished eating with hunger 0
Attendee 4 finished eating with hunger 2
Attendee 0 finished talks with knowledge 1.40 and hunger 12
Attendee 1 finished talks with knowledge 2.92 and hunger 7
Attendee 2 finished talks with knowledge 4.31 and hunger 12
Attendee 3 finished talks with knowledge 1.74 and hunger 11
Attendee 4 finished talks with knowledge 2.08 and hunger 7
Attendee 0 finished eating with hunger 8
Attendee 1 finished eating with hunger 0
Attendee 2 finished eating with hunger 1
Attendee 3 finished eating with hunger 6
Attendee 4 finished eating with hunger

Now, we will model the buffet in more detail using resource!

In [102]:
# config
DURATION_EAT = 3
#   only one person can get food at a time
BUFFET_SLOTS = 1

In [103]:
def attendee(env, name, buffet, knowledge=0, hunger=0):
    # repeat sessions
    while True:
        # visit talks
        for i in range(TALKS_PER_SESSION):
            knowledge += randint(0,3) / (1+hunger) # the more hungry the less knowledge
            hunger += randint(1,4)

            yield env.timeout(TALK_LENGTH)

        print(f'Attendee {name} finished talks with knowledge {knowledge:.2f} and hunger {hunger}')

        # go to buffet
        start = env.now
        # generate a request event
        with buffet.request() as req: # use with statements so that resources are automatically released, otherwise resources need to be released manually
            # yield a condition event to wait either until we get to the buffet 
            # or at least until 12 mins of the break has passed 
            # (otherwise we would not have time to eat)
            yield req | env.timeout(BREAK_LENGTH - DURATION_EAT) # wait for access
            time_left = BREAK_LENGTH - (env.now - start)

            if req.triggered:
                food = min(randint(3,12), time_left) # less time -> less food
                yield env.timeout(DURATION_EAT)
                hunger -= min(food, hunger)
                time_left -= DURATION_EAT
                print(f"Attendee {name} finished eating with hunger {hunger}")
            else:
                hunger -= 1 # penalty for not eating
                print(f"Attendee {name} didn't make it to the buffet, hunger is now {hunger}")

        # wait until break is over
        yield env.timeout(time_left)

# run simulation
env = simpy.Environment()
# create buffet resource
buffet = simpy.Resource(env, capacity=BUFFET_SLOTS)
for i in range(5): 
    env.process(attendee(env, i, buffet))
env.run(until=220)

Attendee 0 finished talks with knowledge 0.25 and hunger 6
Attendee 1 finished talks with knowledge 2.40 and hunger 9
Attendee 2 finished talks with knowledge 0.76 and hunger 10
Attendee 3 finished talks with knowledge 3.00 and hunger 10
Attendee 4 finished talks with knowledge 0.54 and hunger 10
Attendee 0 finished eating with hunger 0
Attendee 1 finished eating with hunger 4
Attendee 2 finished eating with hunger 6
Attendee 3 finished eating with hunger 4
Attendee 4 didn't make it to the buffet, hunger is now 9
Attendee 0 finished talks with knowledge 2.25 and hunger 8
Attendee 1 finished talks with knowledge 3.29 and hunger 14
Attendee 2 finished talks with knowledge 1.24 and hunger 12
Attendee 3 finished talks with knowledge 3.78 and hunger 12
Attendee 4 finished talks with knowledge 0.86 and hunger 18
Attendee 0 finished eating with hunger 0
Attendee 1 finished eating with hunger 10
Attendee 2 finished eating with hunger 3
Attendee 3 finished eating with hunger 6
Attendee 4 didn't

### Priority resource

https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#priorityresource

As you may know from the real world, not every one is equally important. To map that to SimPy, there’s the `PriorityResource`. This subclass of Resource lets requesting processes provide a `priority` for each request. More important requests will gain access to the resource earlier than less important ones. Priority is expressed by integer numbers; *smaller numbers mean a higher priority*

In [104]:
def attendee(env, name, buffet, knowledge=0, hunger=0):
    # repeat sessions
    while True:
        # visit talks
        for i in range(TALKS_PER_SESSION):
            knowledge += randint(0,3) / (1+hunger) # the more hungry the less knowledge
            hunger += randint(1,4)
            prio = randint(0,2)

            yield env.timeout(TALK_LENGTH)

        print(f'Attendee {name} finished talks at {env.now} with knowledge {knowledge:.2f} and hunger {hunger}')

        # go to buffet
        start_break = env.now
        # generate a request event
        with buffet.request(priority=prio) as req: # use with statements so that resources are automatically released, otherwise resources need to be released manually
            # yield a condition event to wait either until we get to the buffet 
            # or at least until 12 mins of the break has passed 
            # (otherwise we would not have time to eat)
            print(f'Attendee {name} requesting to eat buffet at {env.now} with priority {prio}')
            yield req | env.timeout(BREAK_LENGTH - DURATION_EAT) # wait for access
            print(f'Attendee {name} got resource at {env.now}')
            time_left = BREAK_LENGTH - (env.now - start_break)

            if req.triggered:
                food = min(randint(3,12), time_left) # less time -> less food
                yield env.timeout(DURATION_EAT)
                hunger -= min(food, hunger)
                time_left -= DURATION_EAT
                print(f"Attendee {name} finished eating with hunger {hunger}")
            else:
                hunger -= 1 # penalty for not eating
                print(f"Attendee {name} didn't make it to the buffet, hunger is now {hunger}")

        # wait until break is over
        yield env.timeout(time_left)

# run simulation
env = simpy.Environment()
# create buffet resource
buffet = simpy.PriorityResource(env, capacity=BUFFET_SLOTS)
for i in range(3): 
    env.process(attendee(env, i, buffet))
env.run(until=220)

Attendee 0 finished talks at 90 with knowledge 1.45 and hunger 6
Attendee 0 requesting to eat buffet at 90 with priority 0
Attendee 1 finished talks at 90 with knowledge 1.03 and hunger 7
Attendee 1 requesting to eat buffet at 90 with priority 2
Attendee 2 finished talks at 90 with knowledge 0.89 and hunger 10
Attendee 2 requesting to eat buffet at 90 with priority 1
Attendee 0 got resource at 90
Attendee 0 finished eating with hunger 0
Attendee 2 got resource at 93
Attendee 2 finished eating with hunger 5
Attendee 1 got resource at 96
Attendee 1 finished eating with hunger 0
Attendee 0 finished talks at 195 with knowledge 4.87 and hunger 9
Attendee 0 requesting to eat buffet at 195 with priority 0
Attendee 2 finished talks at 195 with knowledge 1.32 and hunger 11
Attendee 2 requesting to eat buffet at 195 with priority 1
Attendee 1 finished talks at 195 with knowledge 1.78 and hunger 8
Attendee 1 requesting to eat buffet at 195 with priority 2
Attendee 0 got resource at 195
Attendee 0

https://stackoverflow.com/questions/23651193/simpy-3-0-4-setting-resource-priority

When the resource is empty, it is based on first come first serve basis. The remaining processes will have to queue to get the resource. 

### Preemptive resource

https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#preemptiveresource

PreemptiveResource inherits from PriorityResource and adds a *preempt flag* (that defaults to True) to `request()`. By setting this to `False` (`resource.request(priority=x, preempt=False)`), a process can decide to not preempt another resource user. It will still be put in the queue according to its priority, though.

In [106]:
def attendee(env, name, buffet, knowledge=0, hunger=0):
    # repeat sessions
    while True:
        # visit talks
        for i in range(TALKS_PER_SESSION):
            knowledge += randint(0,3) / (1+hunger) # the more hungry the less knowledge
            hunger += randint(1,4)
            prio = randint(0,2)

            yield env.timeout(TALK_LENGTH)

        print(f'Attendee {name} finished talks at {env.now} with knowledge {knowledge:.2f} and hunger {hunger}')

        # go to buffet
        start_break = env.now
        # generate a request event
        with buffet.request(priority=prio) as req: # use with statements so that resources are automatically released, otherwise resources need to be released manually
            # yield a condition event to wait either until we get to the buffet 
            # or at least until 12 mins of the break has passed 
            # (otherwise we would not have time to eat)
            try:
                print(f'Attendee {name} requesting to eat buffet at {env.now} with priority {prio}')
                yield req | env.timeout(BREAK_LENGTH - DURATION_EAT) # wait for access
                print(f'Attendee {name} got resource at {env.now}')
                time_left = BREAK_LENGTH - (env.now - start_break)
                
                if req.triggered:
                    food = min(randint(3,12), time_left) # less time -> less food
                    yield env.timeout(DURATION_EAT)
                    hunger -= min(food, hunger)
                    time_left -= DURATION_EAT
                    print(f"Attendee {name} finished eating with hunger {hunger}")
                else:
                    hunger -= 1 # penalty for not eating
                    print(f"Attendee {name} didn't make it to the buffet, hunger is now {hunger}")
            
            except simpy.Interrupt as interrupt:
                by = interrupt.cause.by
                usage = env.now - interrupt.cause.usage_since
                print(f'Attendee {name} got preempted by {by} at {env.now} after {usage}')
        
        # wait until break is over
        yield env.timeout(time_left)

# run simulation
env = simpy.Environment()
# create buffet resource
buffet = simpy.PreemptiveResource(env, capacity=BUFFET_SLOTS)
for i in range(3): 
    env.process(attendee(env, i, buffet))
env.run(until=220)

Attendee 0 finished talks at 90 with knowledge 2.50 and hunger 7
Attendee 0 requesting to eat buffet at 90 with priority 0
Attendee 1 finished talks at 90 with knowledge 2.08 and hunger 5
Attendee 1 requesting to eat buffet at 90 with priority 0
Attendee 2 finished talks at 90 with knowledge 3.67 and hunger 8
Attendee 2 requesting to eat buffet at 90 with priority 1
Attendee 0 got resource at 90
Attendee 0 finished eating with hunger 0
Attendee 1 got resource at 93
Attendee 1 finished eating with hunger 0
Attendee 2 got resource at 96
Attendee 2 finished eating with hunger 0
Attendee 0 finished talks at 195 with knowledge 4.90 and hunger 10
Attendee 0 requesting to eat buffet at 195 with priority 2
Attendee 1 finished talks at 195 with knowledge 5.48 and hunger 10
Attendee 1 requesting to eat buffet at 195 with priority 2
Attendee 2 finished talks at 195 with knowledge 7.50 and hunger 6
Attendee 2 requesting to eat buffet at 195 with priority 1
Attendee 0 got preempted by <Process(atte

## Containers 

Containers help you modelling the production and consumption of a homogeneous, undifferentiated bulk. It may either be continuous (like water) or discrete (like apples).

You can use this, for example, to model the gas / petrol tank of a gas station. Tankers increase the amount of gasoline in the tank while cars decrease it.

In [111]:
class GasStation:
    def __init__(self, env):
        self.fuel_dispensers = simpy.Resource(env, capacity=2)
        self.gas_tank = simpy.Container(env, init=100, capacity=1000)
        self.mon_proc = env.process(self.monitor_tank(env))

    def monitor_tank(self, env):
        while True:
            if self.gas_tank.level < 100:
                print(f"Calling tanker at {env.now}")
                env.process(tanker(env, self))
            yield env.timeout(15) 

def tanker(env, gas_station):
    yield env.timeout(10) # need 10 minutes to arrive
    print(f"Tanker arriving at {env.now}")
    amount = gas_station.gas_tank.capacity - gas_station.gas_tank.level
    yield gas_station.gas_tank.put(amount)

def car(name, env, gas_station):
    print(f"Car {name} arriving at {env.now}")
    with gas_station.fuel_dispensers.request() as req:
        yield req
        print(f"Car {name} start refueling at {env.now}")
        yield gas_station.gas_tank.get(40)
        yield env.timeout(5) # 5 min to refuel tank
        print(f"Car {name} done refueling at {env.now}")

def car_generator(env, gas_station):
    for i in range(4):
        env.process(car(i, env, gas_station))
        yield env.timeout(5) # wait 5 min between each car

env = simpy.Environment()
gas_station = GasStation(env)
car_gen = env.process(car_generator(env, gas_station))
env.run(35)

Car 0 arriving at 0
Car 0 start refueling at 0
Car 1 arriving at 5
Car 0 done refueling at 5
Car 1 start refueling at 5
Car 2 arriving at 10
Car 1 done refueling at 10
Car 2 start refueling at 10
Calling tanker at 15
Car 3 arriving at 15
Car 3 start refueling at 15
Tanker arriving at 25
Car 2 done refueling at 30
Car 3 done refueling at 30


Containers allow you to retrieve their current `level` as well as their `capacity`(see GasStation.monitor_tank() and tanker()). You can also access the list of waiting events via the `put_queue` and `get_queue` attributes (similar to Resource.queue).

## Stores 

Using Stores you can model the production and consumption of concrete objects (in contrast to the rather abstract “amount” stored in containers). A single Store can even contain multiple types of objects.

# Simulating a Single Server Queuing System with Python

https://medium.com/analytics-vidhya/simulating-a-single-server-queuing-system-in-python-f8e32578749f

# Simulating two-sided mobility platforms with MaaSSim

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9182263/